In [ ]:
!pip install koreanize-matplotlib
!pip install konlpy
!pip install gensim

## 실습: word2vec 실습
word2vec 의미를 이해한다
- skip-gram
- cbow

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import koreanize_matplotlib
from collections import Counter
from konlpy.tag import Okt

from gensim.models import Word2Vec
from sklearn.manifold import TSNE
import numpy as np
import time

In [3]:
df=pd.read_csv("./emergency_call_binary_tokens.csv", encoding="utf-8-sig")
df.head()

Unnamed: 0  id                    document label  \
0           0   1    골목길에서 싸움이 났어요 다친 사람이 있어요    긴급   
1           1   2      교통사고가 발생했는데 상대방이 도주했어요    긴급   
2           2   3      집 근처에서 폭죽 소리가 나고 시끄러워요    경미   
3           3   4  술집에서 손님끼리 다투고 있어요 경찰이 필요해요    긴급   
4           4   5         차에 누가 돌을 던졌어요 잡아주세요    경미   

                                    tokens  
0         ['골목길', '싸움', '나다', '다치다', '사람']  
1  ['교통사고', '발생', '하다', '상대방', '도주', '하다']  
2    ['집', '근처', '폭죽', '소리', '나다', '시끄럽다']  
3  ['술집', '손님', '끼리', '다투다', '경찰', '필요하다']  
4            ['차', '누가', '돌', '던지다', '잡다']

In [ ]:
import ast                                                  # 안전한 문자열 → 리스트 변환
df['tokens'] = df['tokens'].apply(ast.literal_eval)         # 문자열 → 실제 리스트로 변환
list_tokens = list(df['tokens'])                            # 토큰 리스트 모음 (문장 수 만큼)

In [ ]:
print(df['tokens'][0])

In [ ]:
start = time.time()
model_skip_gram  = Word2Vec(list_tokens, vector_size=100, window=3, min_count=1, sg=1)
elapsed_time = time.time() - start
print(f"[Skip-gram] 학습 시간: {elapsed_time:.4f}초")

In [ ]:
target_word = '도주'
print('target_word:',target_word)
print(f"\n'{target_word}' 벡터:\n", model_skip_gram.wv[target_word])

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np

def visualize_word2vec_single(model, base_word='배우', topn=10,
                              model_label='Model', color='blue', random_state=42):
    """
    하나의 Word2Vec 모델에서 기준 단어(base_word)와 그 이웃 단어(topn)를
    t-SNE로 2차원에 투영하여 산점도로 시각화합니다.

    Parameters
    ----------
    model : gensim.models.Word2Vec
        시각화할 Word2Vec 모델 (CBOW 또는 Skip-gram 아무거나 1개)
    base_word : str
        기준 단어
    topn : int
        most_similar에서 가져올 이웃 단어 수
    model_label : str
        범례에 표시할 모델 라벨 (예: 'CBOW', 'Skip-gram')
    color : str
        점과 라벨에 사용할 색상
    random_state : int
        t-SNE 난수 고정값
    """

    # 1) 기준 단어 존재 확인
    if base_word not in model.wv:
        print(f"'{base_word}' 단어가 모델의 vocabulary에 있어야 합니다.")
        return

    # 2) 이웃 단어 추출 (기준 단어 + topn개)
    words = [base_word] + [w for w, _ in model.wv.most_similar(base_word, topn=topn)]
    vecs = np.array([model.wv[w] for w in words])

    # 3) t-SNE 차원 축소 (perplexity는 샘플 수보다 작아야 함)
    n_samples = len(vecs)
    perplexity = max(2, min(30, n_samples - 1))  # 너무 작을 때 오류 방지
    tsne = TSNE(
        n_components=2,
        random_state=random_state,
        perplexity=perplexity,
        init='pca',
        learning_rate=100
    )
    reduced = tsne.fit_transform(vecs)

    # 4) 시각화
    plt.figure(figsize=(6, 5))
    for i, word in enumerate(words):
        x, y = reduced[i]
        if word == base_word:
            plt.scatter(x, y, color=color, marker='*', s=150, label=f'{model_label} (base_word)')
        else:
            plt.scatter(x, y, color=color, marker='o', s=30, alpha=0.7)
        plt.annotate(word, (x + 0.9, y + 0.9), fontsize=9, color=color)

    plt.title(f"{model_label} Word2Vec TSNE (기준 단어: '{base_word}')", fontsize=10)
    #plt.legend(frameon=False)
    plt.grid(False)
    plt.tight_layout()
    plt.show()


In [ ]:
base_word = '도주'

In [ ]:
# CBOW만 시각화
model_cbow = Word2Vec(list_tokens, vector_size=100, window=3, min_count=1, sg=0)
visualize_word2vec_single(model_cbow, base_word=base_word, topn=6, model_label='CBOW', color='black')

In [ ]:
# Skip-gram만 시각화
model_sg = Word2Vec(list_tokens, vector_size=100, window=3, min_count=1, sg=1)
visualize_word2vec_single(model_sg, base_word=base_word,
                          topn=6, model_label='Skip-gram', color='black')

  Word2Vec 모델의 전체 vocabulary 또는 상위 N개 단어를 t-SNE로 시각화합니다.

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np

def visualize_word2vec_all(model, topn=None, random_state=42):
    # 1) 시각화 대상 단어 선정
    vocab = model.wv.index_to_key
    if topn is not None:
        vocab = vocab[:topn]

    print(f"[시각화] 단어 수: {len(vocab)}개")

    # 2) 벡터 추출
    vectors = np.array([model.wv[word] for word in vocab])

    # 3) t-SNE
    perplexity = max(5, min(30, len(vocab) - 1))  # 안전한 설정
    tsne = TSNE(
        n_components=2,
        perplexity=perplexity,
        random_state=random_state,
        init='pca',
        learning_rate=100
    )
    reduced = tsne.fit_transform(vectors)

    # 4) 시각화
    plt.figure(figsize=(10, 8))
    plt.scatter(reduced[:, 0], reduced[:, 1], s=25, alpha=0.7, color='black')

    for i, word in enumerate(vocab):
        plt.annotate(word, (reduced[i, 0] + 0.3, reduced[i, 1] + 0.3), fontsize=9)

    plt.title("Word2Vec (CBOW) 전체 단어 임베딩 시각화 (t-SNE)", fontsize=12)
    plt.grid(False)
    plt.tight_layout()
    plt.show()


In [ ]:
model_cbow = Word2Vec(list_tokens, vector_size=100, window=3, min_count=1, sg=0)

# 전체 단어 또는 상위 30개 단어 시각화
visualize_word2vec_all(model_cbow, topn=None)  # 전체를 보려면 topn=None

In [ ]:
model_skip_gram = Word2Vec(list_tokens, vector_size=100, window=3, min_count=1, sg=1)

# 전체 단어 또는 상위 30개 단어 시각화
visualize_word2vec_all(model_skip_gram, topn=None)  # 전체를 보려면 topn=None